In [178]:
import pandas as pd
import numpy as np
from pathlib import Path

WEIGHT_MEAN_PRICE = '成交均价滑差'
TRADE_VOLUME = '成交手数'

pd.set_option('display.float_format', '{:.2f}'.format)

def calc_mean_price_and_vol(df):
    df = df.copy()
    turnover = np.dot(df.price, df.volume)
    volume = df.volume.sum()
    mean_price = turnover / volume
    return pd.Series({WEIGHT_MEAN_PRICE: mean_price, TRADE_VOLUME: volume})
    

In [179]:
p1 = Path(r'D:\vnpy-2.0.7\examples\rohan_follow\.vntrader\trade')
p2 = Path(r'D:\vnpy-2.0.7\examples\ruida_follow\.vntrader\trade')
p3 = Path(r'D:\vnpy-2.0.7\examples\guomao_follow\.vntrader\trade')
folders = [p1, p2, p3]

group_files = [list(p.glob('*.csv')) for p in folders]
files = [f for g in group_files for f in g]
files

[WindowsPath('D:/vnpy-2.0.7/examples/rohan_follow/.vntrader/trade/trade_20191213.csv'),
 WindowsPath('D:/vnpy-2.0.7/examples/rohan_follow/.vntrader/trade/trade_20191216.csv'),
 WindowsPath('D:/vnpy-2.0.7/examples/ruida_follow/.vntrader/trade/trade_20191213.csv'),
 WindowsPath('D:/vnpy-2.0.7/examples/ruida_follow/.vntrader/trade/trade_20191216.csv'),
 WindowsPath('D:/vnpy-2.0.7/examples/guomao_follow/.vntrader/trade/trade_20191213.csv'),
 WindowsPath('D:/vnpy-2.0.7/examples/guomao_follow/.vntrader/trade/trade_20191216.csv')]

In [180]:
df = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)
df.head()

,date,direction,dt,exchange,gateway_name,offset,orderid,price,source_account,symbol,time,tradeid,volume,vt_orderid,vt_tradeid
0,20191213,空,20191213 09:30:16,CFFEX,ROHON,平,1576152660_6280_8724,3940.20,918068,IF1912,09:30:16,2793,1,ROHON.1576152660_6280_8724,ROHON. 2793
1,20191213,空,20191213 09:30:16,CFFEX,ROHON,平,1576152660_6280_8724,3940.20,918068,IF1912,09:30:16,2794,3,ROHON.1576152660_6280_8724,ROHON. 2794
2,20191213,空,20191213 09:30:16,CFFEX,ROHON,平,1576152660_6280_8724,3940.00,918068,IF1912,09:30:16,2795,1,ROHON.1576152660_6280_8724,ROHON. 2795
3,20191213,空,20191213 09:34:42,CFFEX,ROHON,平,1576152660_6280_8725,3940.40,918068,IF1912,09:34:42,2875,4,ROHON.1576152660_6280_8725,ROHON. 2875
4,20191213,空,20191213 10:01:34,CFFEX,ROHON,开,1576152660_6413_18025,3955.00,918068,IF1912,10:01:34,3435,2,ROHON.1576152660_6413_18025,ROHON. 3435


In [181]:
# 分组聚合计算成交均价
res = df.groupby(['source_account', 'gateway_name', 'direction']).apply(calc_mean_price_and_vol).unstack().T
res

source_account   811887           918068           10032518        
gateway_name          CTP     RPC    ROHON     RPC      CTP     RPC
       direction                                                   
成交均价滑差 多          4337.53 4338.48  3971.44 3971.67  4386.52 4386.97
       空          4493.91 4493.58  3966.89 3966.66  4168.64 4168.17
成交手数   多            17.00   17.00    40.00   40.00    23.00   23.00
       空            82.00   82.00    49.00   49.00    81.00   81.00

In [182]:
# 计算滑差
res['811887S'] = res.iloc[:, 1] - res.iloc[:, 0]
res['918068S'] = res.iloc[:, 3] - res.iloc[:, 2]
res['10032518S'] = res.iloc[:, 5] - res.iloc[:, 4]
res['all'] = 0
res

source_account    811887          918068         10032518         811887S  \
gateway_name         CTP     RPC   ROHON     RPC      CTP     RPC           
       direction                                                            
成交均价滑差 多         4337.53 4338.48 3971.44 3971.67  4386.52 4386.97    0.95   
       空         4493.91 4493.58 3966.89 3966.66  4168.64 4168.17   -0.33   
成交手数   多           17.00   17.00   40.00   40.00    23.00   23.00    0.00   
       空           82.00   82.00   49.00   49.00    81.00   81.00    0.00   

source_account   918068S 10032518S all  
gateway_name                            
       direction                        
成交均价滑差 多            0.23      0.44   0  
       空           -0.24     -0.48   0  
成交手数   多            0.00      0.00   0  
       空            0.00      0.00   0

In [183]:
# 根据成交量计算权重
long_vol = res.iloc[2, :].sum() / 2
short_vol = res.iloc[3, :].sum() / 2
# print(long_vol, short_vol)
long_weights = [res.iloc[2, 1] / long_vol, res.iloc[2, 3] / long_vol, res.iloc[2, 5] / long_vol]
short_weights = [res.iloc[3, 1] / short_vol, res.iloc[3, 3] / short_vol, res.iloc[3, 5] / short_vol]
# long_weights, short_weights, sum(long_weights), sum(short_weights)

In [184]:
# 计算加权平均跟踪滑差
res.iloc[0, 9] = np.dot(long_weights, res.iloc[0, 6:9])
res.iloc[1, 9] = np.dot(short_weights, res.iloc[1, 6:9])
res.iloc[2, 9] = long_vol
res.iloc[3, 9] = short_vol
res

source_account    811887          918068         10032518         811887S  \
gateway_name         CTP     RPC   ROHON     RPC      CTP     RPC           
       direction                                                            
成交均价滑差 多         4337.53 4338.48 3971.44 3971.67  4386.52 4386.97    0.95   
       空         4493.91 4493.58 3966.89 3966.66  4168.64 4168.17   -0.33   
成交手数   多           17.00   17.00   40.00   40.00    23.00   23.00    0.00   
       空           82.00   82.00   49.00   49.00    81.00   81.00    0.00   

source_account   918068S 10032518S    all  
gateway_name                               
       direction                           
成交均价滑差 多            0.23      0.44   0.44  
       空           -0.24     -0.48  -0.37  
成交手数   多            0.00      0.00  80.00  
       空            0.00      0.00 212.00

In [185]:
# 调整要展示的数据
res.iloc[2, 6:9] = res.iloc[2, [0, 2, 4]].values
res.iloc[3, 6:9] = res.iloc[3, [0, 2, 4]].values
n_res = res.iloc[:, 6:].copy()
n_res.columns.names = ['account', '']
n_res

account          811887S 918068S 10032518S    all
                                                 
       direction                                 
成交均价滑差 多            0.95    0.23      0.44   0.44
       空           -0.33   -0.24     -0.48  -0.37
成交手数   多           17.00   40.00     23.00  80.00
       空           82.00   49.00     81.00 212.00

In [186]:
# df2 = df.groupby(['source_account', 'gateway_name', 'direction'])['price'].agg(['mean', 'count']).unstack().T
# df2['slippage'] =  np.abs(df2['RPC'] - df2['ROHON'])
# df2
